In [ ]:
# Plotting the EVENT DAYS of Nequick2 Model with Real data with Imputed by SAITS Model (Interactive)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os
import ipywidgets as widgets
from IPython.display import display

# ==== USER INPUT PATHS ====
nequick2_path = "/content/drive/MyDrive/ISMR-SASTRA/EVENT_MONTHS_NEQUICK_DATA.xlsx"
real_path = "/content/drive/MyDrive/ISMR-SASTRA/EVENT MONTHS-REAL DATA.xlsx"
model_path = "/content/drive/MyDrive/ISMR-SASTRA/saits_vtec_model.pkl"

# ==== LOAD TRAINED SAITS MODEL ====
with open(model_path, 'rb') as f:
    saits_model = pickle.load(f)

# ==== LOAD BOTH DATASETS ====
nequick2_xls = pd.ExcelFile(nequick2_path)
real_xls = pd.ExcelFile(real_path)

# Mapping event days to month sheets
event_days = {
    "20-Mar": ("MARCH", 20),
    "22-Jun": ("JUNE", 22),
    "22-Sep": ("SEPTEMBER", 22),
    "21-Dec": ("DECEMBER", 21),
}

# ==== FUNCTION TO EXTRACT AND IMPUTE REAL DATA ====
def get_vtec_from_sheet(xls, sheet, day):
    df = pd.read_excel(xls, sheet_name=sheet, header=None)
    # Columns correspond to days; rows to hours (1–24)
    vtec = df.iloc[:, day - 1].to_numpy(dtype=np.float32)
    if len(vtec) < 24:
        vtec = np.pad(vtec, (0, 24 - len(vtec)), constant_values=np.nan)
    elif len(vtec) > 24:
        vtec = vtec[:24]
    return vtec

# ==== FUNCTION TO PLOT VTEC FOR SELECTED DAY ====
def plot_vtec(day_label):
    sheet, day = event_days[day_label]

    # ---- Load NEQUICK2 and REAL data ----
    vtec_nequick2 = get_vtec_from_sheet(nequick2_xls, sheet, day)
    vtec_real = get_vtec_from_sheet(real_xls, sheet, day)

    # ---- Impute missing values in real data using SAITS ----
    mask = ~np.isnan(vtec_real)
    pred = saits_model.predict({"X": vtec_real.reshape(1, 24, 1)})
    imputed = pred["imputation"].flatten()
    vtec_real_imputed = np.where(mask, vtec_real, imputed)

    # ---- Plot ----
    plt.figure(figsize=(12,6))
    hours = range(1, 25)
    plt.plot(hours, vtec_nequick2, color='red', lw=2, label='NEQUICK2 Model')
    plt.plot(hours, vtec_real_imputed, color='blue', lw=2, label='Real VTEC (SAITS Imputed)')

    plt.xlabel("Hour (1–24)")
    plt.ylabel("VTEC (TECU)")
    plt.title(f"VTEC Comparison - {day_label}")
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.xticks(hours)

    # Y-axis range and ticks
    plt.ylim(0, max(np.max(vtec_nequick2), np.max(vtec_real_imputed)) + 20)
    plt.yticks(np.arange(0, int(max(np.max(vtec_nequick2), np.max(vtec_real_imputed))) + 21, 10))

    plt.tight_layout()
    plt.show()

# ==== DROPDOWN WIDGET ====
event_dropdown = widgets.Dropdown(
    options=list(event_days.keys()),
    description='Select Day:',
    value='20-Mar'
)

widgets.interact(plot_vtec, day_label=event_dropdown)